In [1]:
import dask.dataframe as dd

In [2]:
#read two of the reqiured csv files

company_ds = dd.read_csv('free_company_dataset.csv', on_bad_lines='skip')
recall_ds = dd.read_csv('recalls-market-withdrawals-safety-alerts.csv')


In [3]:
na_states = [
    #usa
    'alabama',
    'alaska',
    'arizona',
    'arkansas',
    'california',
    'colorado',
    'connecticut',
    'delaware',
    'florida',
    'georgia',
    'hawaii',
    'idaho',
    'illinois',
    'indiana',
    'iowa',
    'kansas',
    'kentucky',
    'louisiana',
    'maine',
    'maryland',
    'massachusetts',
    'michigan',
    'minnesota',
    'mississippi',
    'missouri',
    'montana',
    'nebraska',
    'nevada',
    'new hampshire',
    'new jersey',
    'new mexico',
    'new york',
    'north carolina',
    'north dakota',
    'ohio',
    'oklahoma',
    'oregon',
    'pennsylvania',
    'rhode island',
    'south carolina',
    'south dakota',
    'tennessee',
    'texas',
    'utah',
    'vermont',
    'virginia',
    'washington',
    'west virginia',
    'wisconsin',
    'wyoming',
    #canada
    'alberta',
    'british columbia',
    'manitoba',
    'new brunswick',
    'newfoundland and labrador',
    'nova scotia',
    'ontario',
    'prince edward island',
    'quebec',
    'saskatchewan',
    'northwest territories',
    'nunavut',
    'yukon',
    #mexico
    'aguascalientes',
    'baja california',
    'baja california sur',
    'campeche',
    'chiapas',
    'chihuahua',
    'coahuila',
    'colima',
    'durango',
    'guanajuato',
    'guerrero',
    'hidalgo',
    'jalisco',
    'mexico state',
    'michoacan',
    'morelos',
    'nayarit',
    'nuevo leon',
    'oaxaca',
    'puebla',
    'queretaro',
    'quintana roo',
    'san luis potosi',
    'sinaloa',
    'sonora',
    'tabasco',
    'tamaulipas',
    'tlaxcala',
    'veracruz',
    'yucatan',
    'zacatecas'
]

In [4]:
#removing accents from the region column for better matching

from unidecode import unidecode

company_ds['region_no_accents'] = company_ds['region'].apply(lambda x: unidecode(str(x)).lower().strip())
na_regions_processed = [unidecode(region).lower().strip() for region in na_states]

c:\Users\khushaal.a.chaudhary\Anaconda3\envs\geo_gensim_env\lib\site-packages\dask\dataframe\core.py:4419: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('region', 'object'))

  warnings.warn(meta_warning(meta))


In [5]:
#filtering NA records from the company dataset

na_records = company_ds[company_ds['region_no_accents'].isin(na_regions_processed)]

In [6]:
#preprocessing the company name columns in all three datasets due to descripancies in their naming conventions

na_records['id_processed'] = na_records['id'].str.lower().str.replace('[^a-z0-9]','').str.strip()
del company_ds

recall_ds['Brand-Names_processed'] = recall_ds['Brand-Names'].str.replace('[^a-z0-9]','').str.strip()


c:\Users\khushaal.a.chaudhary\Anaconda3\envs\geo_gensim_env\lib\site-packages\dask\dataframe\accessor.py:96: FutureWarning: The default value of regex will change from True to False in a future version.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)
c:\Users\khushaal.a.chaudhary\Anaconda3\envs\geo_gensim_env\lib\site-packages\dask\dataframe\accessor.py:96: FutureWarning: The default value of regex will change from True to False in a future version.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


In [7]:
#performing 1st outer join

company_recall_merged = na_records.merge(recall_ds, left_on='id_processed', right_on='Brand-Names_processed', how='outer')


In [8]:
#deleting old DFs which are not required & reading the 3rd csv file

del na_records
del recall_ds
violations_ds = dd.read_csv('Data_Download_ECHO_10_17_2023_652eac22dbb06.csv',dtype={'RegistryID': 'object',
       'SupOver80CountUsDisp': 'object'})

violations_ds['FacName_processed'] = violations_ds['FacName'].str.replace('[^a-z0-9]','').str.strip()


c:\Users\khushaal.a.chaudhary\Anaconda3\envs\geo_gensim_env\lib\site-packages\dask\dataframe\accessor.py:96: FutureWarning: The default value of regex will change from True to False in a future version.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


In [9]:
#performing final merge operation

all_merged = company_recall_merged.merge(violations_ds, left_on='id_processed', right_on='FacName_processed', how='outer')

In [10]:
# result=all_merged.compute()

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
#saving the results in a csv file
#Note that this is the point at which we get a memory error

all_merged.to_csv('merged_output.csv',index=False, single_file=True)

MemoryError: Unable to allocate 17.7 GiB for an array with shape (18, 132165827) and data type object